### mediapipe 랜드마크, bbox의 좌표(x1, y1, x2, y2), bbox의 ratio, bbox의 class, 속도 학습
* 시퀀스를 기반으로 클래스 우선 분류, bbox의 ratio와 speed로 최종 클래스 결정
* mediapipe의 랜드마크, YOLO bbox의 좌표, bbox의 비율, 상체의 속도(머리, 양쪽 어깨) 학습
* 비교를 위해 정규화하지 않은 .pt파일도 생성

In [50]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import mediapipe as mp
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, confusion_matrix
from scipy.spatial import distance
from tqdm import tqdm

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

def process_landmarks(landmarks, bbox_ratio):
    selected_landmarks = landmarks[LANDMARKS]   # 지정된 랜드마크 선택
    landmark_features = selected_landmarks[:, :2].flatten()  # (x,y) 좌표
    
    # 바운딩 박스 비율 및 속도 정보 추가 
    speed_feature = np.array([0])  # 속도

    # 모든 특성을 결합하여 총 특성 생성
    features = np.concatenate([landmark_features, [bbox_ratio], speed_feature])
    
    return features

def calculate_head_upper_body_speed(current_frame, prev_frame):
    h = np.array([current_frame['landmark_0']['x'], current_frame['landmark_0']['y']])
    l = np.array([current_frame['landmark_11']['x'], current_frame['landmark_11']['y']])
    r = np.array([current_frame['landmark_12']['x'], current_frame['landmark_12']['y']])
    
    prev_h = np.array([prev_frame['landmark_0']['x'], prev_frame['landmark_0']['y']])
    prev_l = np.array([prev_frame['landmark_11']['x'], prev_frame['landmark_11']['y']])
    prev_r = np.array([prev_frame['landmark_12']['x'], prev_frame['landmark_12']['y']])
    
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3
    
    return distance.euclidean(center_new, center_prev)

def calculate_and_draw_bbox(frame, landmarks):
    x_coordinates = landmarks[:, 0]
    y_coordinates = landmarks[:, 1]
    
    x1 = max(0, int(np.min(x_coordinates)))
    y1 = max(0, int(np.min(y_coordinates)))
    x2 = min(frame.shape[1], int(np.max(x_coordinates)))
    y2 = min(frame.shape[0], int(np.max(y_coordinates)))

    # 바운딩 박스를 조금 더 넓게 조정 (각 방향으로 패딩 추가)
    padding = 50
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)

    # 바운딩 박스 비율 계산
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')  # 높이가 0일 경우 무한대로 설정

    return (x1, y1), (x2, y2), bbox_ratio

class FallSequenceDataset(Dataset):
    def __init__(self, json_files, sequence_length=3):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        #self.scaler = StandardScaler()
        self.class_mapping = {0: 'Normal', 1: 'Fall', 2: 'Danger'}
        
        all_landmarks = []
        for json_file in tqdm(json_files, desc="Processing JSON files"):
            try:
                with open(json_file, 'r') as f:
                    data = json.load(f)
                frames = list(data['pose_data'].values())
                fall_start = data.get('fall_start_frame', None)
                fall_end = data.get('fall_end_frame', float('inf'))
                
                for i in range(0, len(frames) - self.sequence_length + 1):
                    sequence = frames[i:i+self.sequence_length]
                    landmarks = []
                    for j, frame in enumerate(sequence):
                        frame_landmarks = []
                        for landmark in LANDMARKS:
                            if f'landmark_{landmark}' not in frame:
                                print(f"Missing landmark {landmark} in frame")
                                continue
                            frame_landmarks.extend([
                                frame[f'landmark_{landmark}']['x'],
                                frame[f'landmark_{landmark}']['y']
                            ])
                        
                        bbox = frame.get('bbox')
                        if bbox:
                            # bbox_ratio 추가
                            bbox_width = bbox['x2'] - bbox['x1']
                            bbox_height = bbox['y2'] - bbox['y1']
                            bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else 1.0                           
                            frame_landmarks.extend([bbox['x1'], bbox['y1'], bbox['x2'], bbox['y2'], bbox_ratio])
                        
                        else:
                            bbox_ratio = 0
                            frame_landmarks.extend([0, 0, 1, 1, bbox_ratio])
                        
                        if j > 0:
                            head_torso_speed = calculate_head_upper_body_speed(sequence[j], sequence[j-1])
                        else:
                            head_torso_speed = 0.0
                        frame_landmarks.append(head_torso_speed)
                        
                        landmarks.append(frame_landmarks)
                    
                    last_frame_index = i + self.sequence_length - 1
                    if fall_start is not None and fall_end is not None:
                        if fall_start <= last_frame_index < fall_end:
                            label = 2  # Danger
                        elif last_frame_index >= fall_end:
                            label = 1  # Fall
                        else:
                            label = 0  # Normal
                    else:
                        label = 0 if frame['class'] == 'Normal' else (1 if frame['class'] == 'Fall' else 2)
                    
                    # bbox_class 설정
                    if bbox_ratio <= 0.7:
                        label = 0  # Normal
                    elif 0.7 < bbox_ratio <= 0.8:
                        label = 2  # Danger
                    else:
                        label = 1  # Fall
                        
                    self.sequences.append(landmarks)
                    self.labels.append(label)
                    all_landmarks.extend(landmarks)
            except Exception as e:
                print(f"Error processing file {json_file}: {e}")
                continue
        
        if not self.sequences:
            raise ValueError("No valid sequences found in the dataset")
        
        #all_landmarks = np.array(all_landmarks)
        #all_landmarks_scaled = self.scaler.fit_transform(all_landmarks)
        
        #for i in range(len(self.sequences)):
        #    start = i * self.sequence_length
        #    end = start + self.sequence_length
        #    self.sequences[i] = all_landmarks_scaled[start:end]
        
        print(f"Total sequences: {len(self.sequences)}")
        print(f"Labels distribution: {np.bincount(self.labels)}")
        print(f"Sequence shape: {len(self.sequences[0])}")
        print(f"Features per frame: {len(self.sequences[0][1])}")

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.FloatTensor(self.sequences[idx]), torch.LongTensor([self.labels[idx]]).squeeze()

class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, num_classes=3, dropout=0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

def train_epoch(model, data_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for sequences, labels in tqdm(data_loader, desc="Training"):
        sequences, labels = sequences.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def validate_epoch(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for sequences, labels in tqdm(data_loader, desc="Validating"):
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            loss = criterion(outputs, labels.view(-1))
            total_loss += loss.item()
    return total_loss / len(data_loader)

def calculate_metrics(model, data_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for sequences, labels in data_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            outputs = model(sequences)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    f1 = f1_score(all_labels, all_preds, average='weighted')
    cm = confusion_matrix(all_labels, all_preds)
    return f1, cm

def main():
    train_json_folder = 'D:\\human_fall\\re_landmark\\addition_yolobbox_json_6'
    val_json_folder = 'D:\\human_fall\\re_landmark\\val_addition_yolobbox_json_6'
    
    train_json_files = [os.path.join(train_json_folder, f) for f in os.listdir(train_json_folder) if f.endswith('.json')]
    val_json_files = [os.path.join(val_json_folder, f) for f in os.listdir(val_json_folder) if f.endswith('.json')]
    
    train_dataset = FallSequenceDataset(train_json_files)
    val_dataset = FallSequenceDataset(val_json_files)
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    
    # 클래스 가중치 계산 (train_dataset의 레이블만 사용)
    train_labels = [label for _, label in train_dataset]
    train_labels = np.array(train_labels, dtype=int)
    unique_classes = np.array([0, 1, 2], dtype=int)  # 모든 가능한 클래스를 명시적으로 지정

    print("train_labels dtype:", train_labels.dtype)
    print("unique_classes dtype:", unique_classes.dtype)
    print("Unique labels in train_labels:", np.unique(train_labels))
    print("unique_classes:", unique_classes)

    class_weights = compute_class_weight('balanced', classes=unique_classes, y=train_labels)
    class_weights = torch.FloatTensor(class_weights).to(device)

    # 손실 함수에 가중치 적용
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    if len(train_dataset) > 0:
        sample_sequence, sample_label = train_dataset[0]
        input_size = sample_sequence.shape[1]
        print(f'Input size: {input_size}')
        model = FallDetectionGRU(input_size).to(device)
    else:
        print("No data available")
        return
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
    
    num_epochs = 150
    best_loss = float('inf')
    patience = 20
    no_improve = 0
    
    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss = validate_epoch(model, val_loader, criterion, device)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        scheduler.step(val_loss)
        
        if val_loss < best_loss:
            best_loss = val_loss
            no_improve = 0
            torch.save(model.state_dict(), 'mediapipe_sensordata_bbox_ratio_speed_except_normalizaion.pt')
            print(f"Best model saved: {best_loss:.4f}")
        else:
            no_improve += 1
            if no_improve >= patience:
                print("Early stopping")
                break
    
    model.load_state_dict(torch.load('mediapipe_sensordata_bbox_ratio_speed_except_normalizaion.pt'))
    train_f1, train_cm = calculate_metrics(model, train_loader, device)
    val_f1, val_cm = calculate_metrics(model, val_loader, device)
    
    # F1 스코어와 혼동 행렬을 파일로 저장
    def save_metrics(train_f1, train_cm, val_f1, val_cm, file_path='mediapipe_sensordata_bbox_ratio_speed_except_normalization.pt.txt'):
        with open(file_path, 'w') as f:
            f.write(f'Train F1: {train_f1:.4f}\n')
            f.write(f'Val F1: {val_f1:.4f}\n')
            f.write(f'Train Confusion Matrix:\n{train_cm}\n')
            f.write(f'Val Confusion Matrix:\n{val_cm}\n')
    
    # F1 스코어와 혼동 행렬을 파일로 저장
    save_metrics(train_f1, train_cm, val_f1, val_cm, file_path='mediapipe_sensordata_bbox_ratio_speed_except_normalization.pt.txt')
    print("저장 완료")
    
    print(f'Train F1: {train_f1:.4f}')
    print(f'Train CM:\n{train_cm}')
    print(f'Val F1: {val_f1:.4f}')
    print(f'Val CM:\n{val_cm}')
    
    print("완료")
    
if __name__ == "__main__":
    main()


### mediapipe의 랜드마크, bbox의 좌표, bbox의 ratio, speed를 학습시켰을 때의 비디오 테스트
* input_size = 28
* sequence_length = 3

In [69]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [70]:
# 랜드마크 인덱스 정의 
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=3, dropout=0.5):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

In [77]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

def process_landmarks(landmarks, bbox_ratio):
    selected_landmarks = landmarks[LANDMARKS]   # 지정된 랜드마크 선택
    landmark_features = selected_landmarks[:, :2].flatten()  # (x,y) 좌표
    
    # 바운딩 박스 비율 및 속도 정보 추가 
    speed_feature = np.array([0])  # 속도

    # 모든 특성을 결합하여 총 특성 생성
    features = np.concatenate([landmark_features, [bbox_ratio], speed_feature])
    
    return features

def calculate_head_upper_body_speed(current_frame, prev_frame):
    h = current_frame[0]
    l = current_frame[1]
    r = current_frame[2]
    
    prev_h = prev_frame[0]
    prev_l = prev_frame[1]
    prev_r = prev_frame[2]
    
    center_new = (h + l + r) / 3
    center_prev = (prev_h + prev_l + prev_r) / 3
    
    return distance.euclidean(center_new, center_prev)

def calculate_and_draw_bbox(frame, landmarks):
    x_coordinates = landmarks[:, 0]
    y_coordinates = landmarks[:, 1]
    
    x1 = max(0, int(np.min(x_coordinates)))
    y1 = max(0, int(np.min(y_coordinates)))
    x2 = min(frame.shape[1], int(np.max(x_coordinates)))
    y2 = min(frame.shape[0], int(np.max(y_coordinates)))

    # 바운딩 박스를 조금 더 넓게 조정 (각 방향으로 패딩 추가)
    padding = 50
    x1 = max(0, x1 - padding)
    y1 = max(0, y1 - padding)
    x2 = min(frame.shape[1], x2 + padding)
    y2 = min(frame.shape[0], y2 + padding)

    # 바운딩 박스 비율 계산
    bbox_width = x2 - x1
    bbox_height = y2 - y1
    bbox_ratio = bbox_width / bbox_height if bbox_height != 0 else float('inf')  # 높이가 0일 경우 무한대로 설정

    return (x1, y1), (x2, y2), bbox_ratio

In [71]:
# 모델 초기화 및 가중치 로드
input_size = 28  # 랜드마크 x,y 좌표 + bbox 좌표 + bbox 비율 + speed 
model = FallDetectionGRU(input_size).to(device)
model.load_state_dict(torch.load('D:\\project\\prjvenv\\GRU\\GRU_pts\\4. mediapipe, sensordata, bbox_ratio, speed\\mediapipe_sensordata_bbox_ratio_speed_except_normalization.pt', map_location=device))
model.eval()

FallDetectionGRU(
  (gru): GRU(28, 64, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=64, out_features=3, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [72]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

In [115]:
# 비디오 파일 경로 설정 및 열기
video_path = "D:\\human_fall\\re_video\\validation\\Y\\00759_O_E_FY_C3.mp4"
cap = cv2.VideoCapture(video_path)

# 시퀀스 길이 설정 (훈련 시 사용한 값과 일치해야 함)
sequence_length = 3  
data_sequence = []

output_path = 'C:\\Users\\user\\Desktop\\prj_sample_vid\\inputsize_28_N_2.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
out = cv2.VideoWriter(output_path, fourcc, 30.0, (1920, 1080))

previous_landmarks = None  # 이전 프레임의 랜드마크를 저장할 변수

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환 및 랜드마크 추출
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        landmarks = []
        
        # 랜드마크 추출 및 표시
        for landmark_idx in LANDMARKS:
            landmark = results.pose_landmarks.landmark[landmark_idx]
            landmarks.append([landmark.x * frame.shape[1], landmark.y * frame.shape[0]])  # 픽셀 좌표로 변환
            
            # 랜드마크를 비디오 프레임에 표시
            cv2.circle(frame, (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])), 5, (0, 255, 0), -1)

        # 랜드마크 배열 변환 및 시퀀스 추가
        landmarks_array = np.array(landmarks).flatten()

        # bbox 좌표 추가 
        if len(landmarks) > 0 : 
            bbox_x1 = int(np.min(landmarks_array[::2]))   # x_min
            bbox_y1 = int(np.min(landmarks_array[1::2]))   # y_min
            bbox_x2 = int(np.max(landmarks_array[::2]))    # x_max
            bbox_y2 = int(np.max(landmarks_array[1::2]))    # y_max
        
            # 바운딩 박스 비율 계산
            bbox_ratio_value = (bbox_x2 - bbox_x1) / (bbox_y2 - bbox_y1) if (bbox_y2 - bbox_y1) != 0 else 0
            
            # predicted_label_id를 기준으로 box_color와 text_color 설정
            box_color = [(0, 255, 0), (0, 255, 255), (0, 0, 255)][predicted_label_id]  # ID로 색상 선택
            text_color = [(0, 255, 0), (0, 255, 255), (0, 0, 255)][predicted_label_id]  # ID로 색상 선택
            label_name = {0: 'Normal', 1: 'Danger', 2: 'Fall'}[predicted_label_id]  # 예측된 클래스 이름
            
            # 예측된 클래스 이름 출력
            label_name = {0: 'Normal', 1: 'Danger', 2: 'Fall'}
            predicted_label_name = label_name[predicted_label_id]
            print(predicted_label_name)
            
            cv2.rectangle(frame, (bbox_x1 - 80, bbox_y1 - 120), (bbox_x2 + 80, bbox_y2 + 80), box_color, 2)
            cv2.putText(frame, f'GRU pred : {predicted_label_name}', (bbox_x1 - 80, bbox_y1 - 140), cv2.FONT_HERSHEY_SIMPLEX, 0.9, text_color, 2)
                        
            # 랜드마크 + bbox 정보 추가
            landmarks_array_with_bbox = np.concatenate((landmarks_array,
                                            [bbox_x1, bbox_y1, bbox_x2, bbox_y2],
                                            [bbox_ratio_value]))

            # 이전 프레임이 있다면 head_torso_speed 계산
            if previous_landmarks is not None:
                selected_indices = [0, 1, 2]  # head(0), leftshoulder(11), rightshoulder(12)
                head_torso_speed = calculate_head_upper_body_speed(landmarks_array.reshape(-1, 2)[selected_indices],
                                                       previous_landmarks.reshape(-1, 2)[selected_indices])
                landmarks_array_with_bbox = np.concatenate((landmarks_array_with_bbox, [head_torso_speed]))
            else:
                head_torso_speed = 0  # 초기 프레임에서는 속도 0으로 설정

            # 시퀀스에 추가된 데이터를 크기가 28에 맞게 조정하기
            if len(data_sequence) < sequence_length - 1:
                data_sequence.append(landmarks_array_with_bbox.tolist()) 
            else:
                data_sequence.append(landmarks_array_with_bbox.tolist() + [head_torso_speed])

        # 시퀀스의 길이를 `28`로 맞추기 위해 추가적인 0 채우기
        if len(data_sequence[-1]) < 28:
            data_sequence[-1] = data_sequence[-1] + [0] * (28 - len(data_sequence[-1]))

        # 시퀀스가 `sequence_length`보다 길어지면 앞의 항목을 제거
        if len(data_sequence) > sequence_length:
            data_sequence.pop(0)

        # 모델 입력 생성
        input_tensor = torch.FloatTensor([data_sequence]).to(device)

        # 모델 추론
        with torch.no_grad():
            outputs = model(input_tensor)
            _, predicted_label = torch.max(outputs, 1)
            predicted_label_id = predicted_label.item()

        # 시퀀스 갱신
        data_sequence.pop(0)
        
        # 이전 랜드마크 업데이트
        previous_landmarks = landmarks_array
            
        # 랜드마크 표시
        mp_drawing.draw_landmarks(frame,
                          results.pose_landmarks,
                          mp_pose.POSE_CONNECTIONS,
                          landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2),
                          connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))

        # 프레임 저장
        resized_frame = cv2.resize(frame, (1920, 1080))
        out.write(resized_frame)


    cv2.imshow('Fall Detection', resized_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Danger
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Fall
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Fall
Danger
Fall
Fall
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Fall
Danger
Fall
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
Danger
D